In [1]:
import os
os.chdir("../")
%pwd

'd:\\DE\\ML OPs\\DSusingMLOps'

In [2]:
from dataclasses import dataclass
from pathlib import Path
import pandas as pd

In [3]:
@dataclass
class ModelTrainerConfig:
    root_dir: Path
    model_name: str
    train_date_path: Path
    test_data_path: Path
    alpha: float
    l1_ratio: float
    target_column: str

In [ ]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml,create_directoriesfrom src.datascience.constants import *
from src.datascience.utils.common import read_yaml,create_directories

In [5]:
class ConfigurationManager():
    def __init__(self, config_filepath=CONFIG_FILE_PATH, 
                                params_filepath= PARAMS_FILE_PATH,
                                schema_filepath=SCHEMA_FILE_PATH ):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_tainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        model_trainer_config = ModelTrainerConfig(
            root_dir=Path(config.root_dir),
            model_name=config.model_name,
            train_date_path=Path(config.train_date_path),
            test_data_path=Path(config.test_data_path),
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=schema.name

            
        )
        return model_trainer_config
        

In [6]:
import pandas as pd
import os
from src.datascience.utils.common import save_json,load_json
from sklearn.linear_model import ElasticNet
import joblib
from src.datascience import logger

In [7]:
class model_trainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train_model(self):
        logger.info("Loading training and test data")
        train_data = pd.read_csv(self.config.train_date_path)
        test_data = pd.read_csv(self.config.test_data_path)

        logger.info("Splitting features and target")
        X_train = train_data.drop(columns=[self.config.target_column], axis=1)
        y_train = train_data[self.config.target_column]

        X_test = test_data.drop(columns=[self.config.target_column], axis=1)
        y_test = test_data[self.config.target_column]

        logger.info("Training the model")
        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(X_train, y_train)

        logger.info("Saving the model")
        os.makedirs(self.config.root_dir, exist_ok=True)
        model_path = os.path.join(self.config.root_dir, self.config.model_name)
        joblib.dump(lr, model_path)
        logger.info(f"Model saved at {model_path}")

In [8]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_instance = model_trainer(config=model_trainer_config)
    model_trainer_instance.train_model()
    logger.info("Model training completed successfully.")
except Exception as e:
    logger.exception(e)
    raise e

[2025-09-13 22:44:11,235: INFO : common: yaml file: config\config.yaml loaded sucessfully]
[2025-09-13 22:44:11,237: INFO : common: yaml file: params.yaml loaded sucessfully]
[2025-09-13 22:44:11,240: INFO : common: yaml file: schema.yaml loaded sucessfully]
[2025-09-13 22:44:11,242: INFO : common: created directory at path :artifacts]
[2025-09-13 22:44:11,243: INFO : 1523863267: Loading training and test data]
[2025-09-13 22:44:11,250: INFO : 1523863267: Splitting features and target]
[2025-09-13 22:44:11,254: INFO : 1523863267: Training the model]
[2025-09-13 22:44:11,258: INFO : 1523863267: Saving the model]
[2025-09-13 22:44:11,260: INFO : 1523863267: Model saved at artifacts\model_trainer\model.joblib]
[2025-09-13 22:44:11,261: INFO : 1556057569: Model training completed successfully.]
